In [ ]:
# Import Labelspark (ls) for working with Labelbox and PySpark SQL for working with Spark
import labelspark as ls
from pyspark.sql import SparkSession
import json

# Retrieve the dataset ID from a widget input
lb_dataset_id = dbutils.widgets.get("dataset_id")

# Retrieve the table path from a widget input
table_name = dbutils.widgets.get("table_path")

# Retrieve the Labelbox API key from a widget input
api_key = dbutils.widgets.get("labelbox_api_key")

# Retrieve the schema map from a widget input
schema_map = json.loads(dbutils.widgets.get("schema_map"))

# Create a SparkSession to work with Spark
spark = SparkSession.builder.getOrCreate()

# Run a SQL command to describe the extended properties of the table and store the result in table_info
table_info = spark.sql(f"DESCRIBE EXTENDED {table_name}")

# Extract the location from the table information and store it in table_path
table_path = table_info.filter("col_name == 'Location'").collect()[0]['data_type']

# Fetch the first 50 rows from the table into a DataFrame
temp_df = spark.sql(f"SELECT * FROM {table_name} LIMIT 50")

# Define a path to save the temporary table. This assumes that `table_path` is a directory.
temp_table_storage_path = f"{table_path}_temp_50_rows"

# Write the DataFrame to the specified path with overwrite mode
temp_df.write.format("delta").mode("overwrite").save(temp_table_storage_path)


# Create a Labelspark client using the provided API key
client = ls.Client(lb_api_key=api_key)

# Use the saved location in the create_data_rows_from_delta_table function
client.create_data_rows_from_delta_table(dataset_id=lb_dataset_id, table_path=temp_table_storage_path, column_mappings=schema_map, skip_duplicates=True, verbose=True, spark=spark)
